In [19]:
import time
import math
import numpy as np
from numpy.fft  import fft2, ifft2
from scipy.signal import convolve2d
import copy

# Data Generation

In [20]:
rows = 7
cols = 7
mu, sigma = 0, math.sqrt(2.0 / (rows * cols))

k_size = 3

filter2d = np.random.normal(mu, sigma, size=(k_size, k_size))

image = np.random.randint(255, size=(rows, cols))

print("filter = ", filter2d)
print("image = ", image)

filter =  [[-0.22191364 -0.12256412 -0.20761138]
 [ 0.21137156  0.06855702 -0.05734186]
 [ 0.05118438 -0.00210179  0.1135126 ]]
image =  [[ 64 119 119 152 206   0 219]
 [247 212 103 225 153 120  65]
 [233 243  19 139  82   2 145]
 [250 181 161 239 242 165 153]
 [198 147 211 189  54 103  18]
 [104 106 122 239 238   6  58]
 [106  66 210 224 148 208  23]]


## Scipy Convolution

Automatically chooses direct or Fourier method based on an estimate
of which is faster (default)

In [21]:
start = time.time()
img = copy.deepcopy(image)
scipy_res = convolve2d(img, filter2d, mode='same', boundary='fill', fillvalue=0)
end = time.time()
print("Finish in %.2f ms" % ((end - start)*1000))
print("scipy_res = ", scipy_res)

Finish in 0.32 ms
scipy_res =  [[-47.77816427 -70.47908758 -73.10520581 -35.77415885 -86.68772376
  -26.41867439 -17.8660453 ]
 [-14.78167312 -47.12533753 -20.12287006  26.4135285    0.42077136
   -1.66107607 -21.07212898]
 [  6.8616215  -69.63626191 -58.23401455 -71.5988559  -86.4606019
  -57.89166757 -29.69720832]
 [ 10.45659322 -46.93337811 -12.48671814 -14.58724141 -15.24344384
   18.67102772 -22.64004351]
 [ 17.11531928  17.90515792 -11.57102867 -65.00483107 -30.41882482
  -20.29382188   5.38143202]
 [  9.00516712 -16.64064782 -10.42922905 -17.88224402 -80.31000488
  -55.46579029 -30.71592114]
 [ 26.42452484  60.66156766  81.96862413  60.12606387  68.20353709
   40.60689336  -9.79112334]]


## My implementation 1

In [22]:
def conv2d(x, f):
    r, c = x.shape
    kr, kc = f.shape
    res = np.zeros(x.shape)
    for i in range(r):
        for j in range(c):
            for m in range(kr):
                mm = kr - m - 1
                for n in range(kc):
                    nn = kc - n - 1
                    ii = i + kr//2 - mm
                    jj = j + kc//2 - nn
                    if 0 <= ii < r and 0 <= jj < c:
                        res[i][j] += x[ii][jj] * f[mm][nn]
    return res
start = time.time()
img = copy.deepcopy(image)
res = conv2d(img, filter2d)
assert np.all((res - scipy_res) < 1e-8)
end = time.time()
print("Finish in %.2f ms" % ((end - start)*1000))
print("my_res = ", res)

Finish in 0.99 ms
my_res =  [[-47.77816427 -70.47908758 -73.10520581 -35.77415885 -86.68772376
  -26.41867439 -17.8660453 ]
 [-14.78167312 -47.12533753 -20.12287006  26.4135285    0.42077136
   -1.66107607 -21.07212898]
 [  6.8616215  -69.63626191 -58.23401455 -71.5988559  -86.4606019
  -57.89166757 -29.69720832]
 [ 10.45659322 -46.93337811 -12.48671814 -14.58724141 -15.24344384
   18.67102772 -22.64004351]
 [ 17.11531928  17.90515792 -11.57102867 -65.00483107 -30.41882482
  -20.29382188   5.38143202]
 [  9.00516712 -16.64064782 -10.42922905 -17.88224402 -80.31000488
  -55.46579029 -30.71592114]
 [ 26.42452484  60.66156766  81.96862413  60.12606387  68.20353709
   40.60689336  -9.79112334]]


## My implementation 2

In [23]:
def conv2d(x, f):
    r, c = x.shape
    kr, kc = f.shape
    res = np.zeros(x.shape)
    for i in range(r):
        for j in range(c):
            # (i, j) is the center position of filter
            for ki in range(-int(kr / 2), int(kr / 2) + 1, 1):
                for kj in range(-int(kc / 2), int(kc / 2) + 1, 1):
                    m = i - ki
                    n = j - kj
                    #print(ki, kj, i, j)
                    if m >= 0 and m < r and n >= 0 and n < c:
                        res[i, j] += x[m, n] * f[ki + int(kr / 2), kj + int(kc / 2)]
    return res
start = time.time()
img = copy.deepcopy(image)
res = conv2d(img, filter2d)
assert np.all((res - scipy_res) < 1e-8)
end = time.time()
print("Finish in %.2f ms" % ((end - start)*1000))
print("my_res = ", res)

Finish in 1.26 ms
my_res =  [[-47.77816427 -70.47908758 -73.10520581 -35.77415885 -86.68772376
  -26.41867439 -17.8660453 ]
 [-14.78167312 -47.12533753 -20.12287006  26.4135285    0.42077136
   -1.66107607 -21.07212898]
 [  6.8616215  -69.63626191 -58.23401455 -71.5988559  -86.4606019
  -57.89166757 -29.69720832]
 [ 10.45659322 -46.93337811 -12.48671814 -14.58724141 -15.24344384
   18.67102772 -22.64004351]
 [ 17.11531928  17.90515792 -11.57102867 -65.00483107 -30.41882482
  -20.29382188   5.38143202]
 [  9.00516712 -16.64064782 -10.42922905 -17.88224402 -80.31000488
  -55.46579029 -30.71592114]
 [ 26.42452484  60.66156766  81.96862413  60.12606387  68.20353709
   40.60689336  -9.79112334]]


## FFT convolution

In [24]:
from numpy.fft  import fft2, ifft2
def np_fftconvolve(A, B):
    return np.real(ifft2(fft2(A)*fft2(B, s=A.shape)))
start = time.time()
img = copy.deepcopy(image)
fft_res = np_fftconvolve(img, filter2d)
assert np.all((res - scipy_res) < 1e-8)
end = time.time()
print("Finish in %.2f ms" % ((end - start)*1000))
print("fft_res = ", res)

Finish in 0.26 ms
fft_res =  [[-47.77816427 -70.47908758 -73.10520581 -35.77415885 -86.68772376
  -26.41867439 -17.8660453 ]
 [-14.78167312 -47.12533753 -20.12287006  26.4135285    0.42077136
   -1.66107607 -21.07212898]
 [  6.8616215  -69.63626191 -58.23401455 -71.5988559  -86.4606019
  -57.89166757 -29.69720832]
 [ 10.45659322 -46.93337811 -12.48671814 -14.58724141 -15.24344384
   18.67102772 -22.64004351]
 [ 17.11531928  17.90515792 -11.57102867 -65.00483107 -30.41882482
  -20.29382188   5.38143202]
 [  9.00516712 -16.64064782 -10.42922905 -17.88224402 -80.31000488
  -55.46579029 -30.71592114]
 [ 26.42452484  60.66156766  81.96862413  60.12606387  68.20353709
   40.60689336  -9.79112334]]
